In [ ]:
# Install required libraries
!pip install PyMuPDF python-docx scikit-learn -q

import os
import fitz  # PyMuPDF
from docx import Document
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from google.colab import files

# Instructions for users
print("""
PDF and DOCX Similarity Checker
------------------------------
1. Upload your PDF or Word (.docx) files (e.g., student assignments).
2. The script will extract text and compare for 95%+ similarity.
3. Results are saved as 'similarity_report.txt' and downloaded automatically.
4. No file contents are stored or displayed for privacy.
""")

# Step 1: Set up directory
upload_dir = '/content/uploads'
os.makedirs(upload_dir, exist_ok=True)

# Step 2: Upload files
print("Please upload your PDF or Word (.docx) files...")
uploaded = files.upload()
for filename, content in uploaded.items():
    if filename.lower().endswith(('.pdf', '.docx')):
        with open(os.path.join(upload_dir, filename), 'wb') as f:
            f.write(content)
    else:
        print(f"Skipped {filename}: Not a PDF or .docx file")

# Step 3: Extract text from files
print("\nExtracting text from files...")
file_texts = []
file_names = []
for filename in os.listdir(upload_dir):
    try:
        text = ''
        file_path = os.path.join(upload_dir, filename)
        if filename.lower().endswith('.pdf'):
            with fitz.open(file_path) as doc:
                for page in doc:
                    page_text = page.get_text()
                    if page_text:
                        text += page_text + '\n'
        elif filename.lower().endswith('.docx'):
            doc = Document(file_path)
            for para in doc.paragraphs:
                if para.text:
                    text += para.text + '\n'
        # Remove student names (customizable)
        text = re.sub(r'(?:Student )?Name: [A-Za-z\s]+', '', text, flags=re.IGNORECASE)
        text = re.sub(r'[A-Za-z\s]+@student\.edu', '', text, flags=re.IGNORECASE)
        if text.strip():
            file_texts.append(text)
            file_names.append(filename)
        else:
            print(f"Skipped {filename}: No extractable text")
    except Exception as e:
        print(f"Error processing {filename}: {e}")

# Step 4: Compare for 95%+ similarity
if len(file_texts) < 2:
    print("\n❌ Not enough valid documents with extractable text.")
else:
    print("\nComparing files for 95%+ similarity...")
    try:
        # TF-IDF Vectorization + Cosine Similarity
        vectorizer = TfidfVectorizer(stop_words='english').fit_transform(file_texts)
        similarity_matrix = cosine_similarity(vectorizer)

        # Threshold for 95%+ similarity
        threshold = 0.95
        results = []
        for i in range(len(file_names)):
            for j in range(i + 1, len(file_names)):
                score = similarity_matrix[i][j]
                if score >= threshold:
                    results.append(f"{file_names[i]} <--> {file_names[j]} : {score:.2f}")

        # Step 5: Save and download results
        output_file = '/content/similarity_report.txt'
        if results:
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write("Similarity Results (95%+ Threshold)\n")
                f.write("----------------------------------\n")
                for result in results:
                    f.write(result + '\n')
            print(f"\nFound {len(results)} pairs with 95%+ similarity. Downloading results...")
            files.download(output_file)
        else:
            print("\nNo pairs with 95%+ similarity found.")
    except Exception as e:
        print(f"Error during comparison: {e}")

# Step 6: Clean up for privacy
print("\nCleaning up temporary files...")
try:
    for file in os.listdir(upload_dir):
        os.remove(os.path.join(upload_dir, file))
    os.rmdir(upload_dir)
except Exception as e:
    print(f"Error cleaning up: {e}")
print("Done! All temporary files deleted.")